In [1]:
import asyncio

In [2]:
async def log_async(task, delay_secs: int = 1):
    print(f"starting task: {task} before delay ({delay_secs} secs)")
    asyncio.sleep(delay_secs)
    print(f"finished pretending to log for {task}")

### Await needed...

In [3]:
tasks  = ["one", "two", "three_async", "four", "five_async", "six"]

for task in tasks:
    if task.endswith("async"):
        # just calling func requires an await somewhere
        log_async(task)
    else:
        print(f"task: {task}")
    

task: one
task: two
task: four
task: six


C:\Users\ChrisProsser\AppData\Local\Temp\ipykernel_12592\3188711805.py:6: RuntimeWarning: coroutine 'log_async' was never awaited
  log_async(task)


### asyncio.run(async_func) doesn't fix alone

> asyncio.run() cannot be called from a running event loop

Could assign variables and await the log_async, but where do we await?
we don't want to delay page being returned to the user

i.e. if we await before `return render_template()` then what was the point in making async, but can't do it after

Could use a constant running loop (outside of req context) to process tasks added to a queue? Like method 2 from https://www.geeksforgeeks.org/how-to-run-two-async-functions-forever-python/?

In [4]:
logger_loop = asyncio.get_event_loop()

def log(task):
    logger_loop.create_task(log_async(task))

def main():
    print("sync 1")
    log("async 2")
    print("sync 3")
    print("sync 4")
    log("async 5")
    
main()   
#logger_loop.run_forever() # This event loop is already running err
                           # coroutine 'sleep' was never awaited waring if excl

                        

sync 1
sync 3
sync 4
starting task: async 2 before delay (1 secs)
finished pretending to log for async 2
starting task: async 5 before delay (1 secs)
finished pretending to log for async 5


C:\Users\ChrisProsser\AppData\Local\Temp\ipykernel_12592\1808806421.py:3: RuntimeWarning: coroutine 'sleep' was never awaited
  asyncio.sleep(delay_secs)


👆 warning + didn't start async tasks until all sync completed - want to start straight away, but not wait before continuing main sync code

Stack Overflow on fire & forget here: https://stackoverflow.com/a/37345564/2012446

I don't think this solves as you need the main func you call log from to be async as well, can't wrap req/resp as func as you need to await at end to ensure any running tasks complete.

### concurrent option

Could do like this: https://stackoverflow.com/a/45843022/2012446
Need to be aware of risk of changed variables after calling func (copy?)